In [ ]:
import pandas as pd

In [ ]:
tbl = pd.read_excel('https://www.dshs.texas.gov/coronavirus/TexasCOVID19DailyCountyCaseCountData.xlsx', skiprows=[0,1]+list(range(257,270)))
#tbl = pd.read_excel('/home/ponyisi/Documents/20200614_covid/Texas COVID-19 Case Count Data by County.xlsx', skiprows=[0,1]+list(range(257,270)))
def mapper(x):
    if not x.startswith('Cases'): return x
    return x[-5:]
tbl=tbl.rename(columns=mapper)
tbl=tbl.set_index('County Name')
tblpop = tbl[['Population']]
#tblpop=tblpop.set_index('County Name')
tblcases = tbl.loc[:,'03-04':]
#tblcases=tblcases.transpose()

In [ ]:
# single day new cases
tbl2=tblcases.diff(axis=1).loc[:,'03-05':]
# 7 day rolling average (to smooth weekly effects) then exponential moving average (for smoothing)
tbl3=tbl2.rolling(7,axis=1).mean().loc[:,'03-13':].ewm(halflife=1,axis=1).mean()
# cases per 100000 population
tbl4=tbl3.div(tblpop['Population'], axis=0)*100000

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
print(list(_ for _ in counties['features'] if (_['id'].startswith('48') and _['properties']['NAME'] == 'Travis')))

In [ ]:
counties['features'][0]

In [ ]:
import plotly.express as px
import copy
txcounties=copy.copy(counties)
txcounties['features']=[_ for _ in counties['features'] if _["id"].startswith('48')]
fig = px.choropleth(tbl4, geojson=txcounties, color=tbl4.columns[-1],
                    locations=tbl4.index, featureidkey="properties.NAME",
                    projection="mercator", range_color=[0,25]
                   )
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# reformat for animation
exploded=tbl4.reset_index().melt(id_vars=['County Name'], var_name='Date', value_name='Rate')
fig = px.choropleth(exploded, geojson=txcounties, color="Rate",
                    locations='County Name', featureidkey="properties.NAME",
                    projection="mercator", range_color=[0,15],
                    animation_group='County Name', animation_frame='Date',
                    labels={'Rate':'Daily new cases/100,000 pop'},
                    width=800, height=700
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# single day change in 7-day rolling average of new cases
tbl3der=tbl3.diff(axis=1).loc[:,'03-14':]
# change in cases per 100000 population
tbl4der=tbl3der.div(tblpop['Population'], axis=0)*100000
# reformatted for plotting
explodedder=tbl4der.reset_index().melt(id_vars=['County Name'], var_name='Date', value_name='Rate')

In [ ]:
figder = px.choropleth(explodedder, geojson=txcounties, color="Rate",
                    locations='County Name', featureidkey="properties.NAME",
                    projection="mercator", range_color=[-5,5],
                       #color_continuous_midpoint=0.,
                    animation_group='County Name', animation_frame='Date',
                    labels={'Rate':'Daily case change'},
                    color_continuous_scale='rdbu_r',
                    width=800, height=700
                   )
figder.update_geos(fitbounds="locations", visible=False)
figder.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
figder.show()

In [ ]:
# absolute number of daily cases (7-day rolling)
exploded_tot=tbl3.reset_index().melt(id_vars=['County Name'], var_name='Date', value_name='Rate')
figtot = px.choropleth(exploded_tot, geojson=txcounties, color="Rate",
                    locations='County Name', featureidkey="properties.NAME",
                    projection="mercator", range_color=[0,350],
                    animation_group='County Name', animation_frame='Date',
                    labels={'Rate':'Daily new cases'},
                    width=800, height=700
                   )
figtot.update_geos(fitbounds="locations", visible=False)
figtot.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
figtot.show()